
# **TP3 : Transfert d’Apprentissage**
Dans ce TP, nous explorons la technique de transfert d’apprentissage (*Transfer Learning*). Cela consiste à réutiliser un modèle pré-entraîné sur un problème similaire pour résoudre une nouvelle tâche.

### **Objectifs**
1. Comprendre le concept de transfert d'apprentissage.
2. Utiliser un modèle pré-entraîné (MobileNetV2).
3. Adapter ce modèle pour classifier les images du jeu de données CIFAR-10.

Le transfert d'apprentissage permet de gagner du temps et des ressources tout en obtenant de meilleures performances, même avec peu de données.



## **1. Chargement et Préparation des Données**
Nous allons utiliser le jeu de données CIFAR-10, qui contient 60 000 images classées en 10 catégories :
- Avion, Automobile, Oiseau, Chat, Cerf, Chien, Grenouille, Cheval, Bateau, Camion.


In [ ]:

# Import des bibliothèques nécessaires
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import matplotlib.pyplot as plt

# Chargement des données CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalisation des images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Encodage des labels en one-hot
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Affichage de quelques exemples
classes = ['Avion', 'Automobile', 'Oiseau', 'Chat', 'Cerf', 'Chien', 'Grenouille', 'Cheval', 'Bateau', 'Camion']
plt.figure(figsize=(10, 5))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(x_train[i])
    plt.title(f"Classe : {classes[y_train[i].argmax()]}")
    plt.axis('off')
plt.show()



## **2. Chargement d’un Modèle Pré-entraîné**
Nous utilisons MobileNetV2, un modèle pré-entraîné sur ImageNet.


In [ ]:

# Chargement de MobileNetV2 pré-entraîné
base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')

# Geler les couches du modèle pré-entraîné
base_model.trainable = False

# Ajout de nouvelles couches pour la classification
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Réduction de la dimension des caractéristiques
    Dense(128, activation='relu'),  # Couche dense
    Dense(10, activation='softmax')  # Couche de sortie pour 10 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Résumé du modèle
model.summary()



## **3. Entraînement du Modèle**
Nous entraînons uniquement les couches ajoutées pour adapter MobileNetV2 au jeu de données CIFAR-10.


In [ ]:

# Entraînement du modèle
history = model.fit(x_train, y_train, epochs=5, validation_split=0.2)



## **4. Visualisation des Performances**
Observons l'évolution de la précision et de la perte pendant l'entraînement.


In [ ]:

# Visualisation des courbes d'entraînement
plt.plot(history.history['accuracy'], label='Précision Entraînement')
plt.plot(history.history['val_accuracy'], label='Précision Validation')
plt.xlabel('Époques')
plt.ylabel('Précision')
plt.legend()
plt.show()



## **5. Évaluation sur les Données de Test**
Nous évaluons les performances du modèle sur les données de test.


In [ ]:

# Évaluation sur les données de test
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Précision sur les données de test : {test_acc:.2f}")



## **6. Visualisation des Prédictions**
Affichons les prédictions pour quelques images.


In [ ]:

# Prédictions sur les premières images de test
predictions = model.predict(x_test[:5])

# Affichage des résultats
for i in range(5):
    plt.imshow(x_test[i])
    plt.title(f"Prediction: {classes[predictions[i].argmax()]}")
    plt.axis('off')
    plt.show()



## **7. Discussion et Questions**
### Biais dans les Modèles Pré-entraînés
MobileNetV2 a été pré-entraîné sur ImageNet, qui contient des biais liés aux données utilisées. 
- Les images proviennent majoritairement de contextes occidentaux.
- Certaines catégories sont sous-représentées.

**Questions** :
1. Quels sont les impacts possibles de ces biais sur des applications spécifiques (par ex. médicales) ?
2. Comment minimiser ces biais dans le transfert d'apprentissage ?

**Exercice :** Remplacez MobileNetV2 par un autre modèle pré-entraîné (par ex. ResNet50 ou InceptionV3) et comparez les résultats.
